<a href="https://colab.research.google.com/github/r-karra/Quantum-AI/blob/main/Experiments_with_QuantumComputing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧭 Traveling Salesperson Problem (TSP) — Python Simulation


In [3]:
# 🧭 Traveling Salesperson Problem (TSP) — Python Simulation

# This example uses **5 cities** placed randomly on a 2D map.
# We’ll compare how three different algorithms find a route visiting all cities:

# 1. **Nearest Neighbour**
# 2. **Multiple Fragment (Greedy edges)**
# 3. **Branch & Bound (simple version)**


import itertools
import math
import random

# 🎯 Generate 5 random "cities" with x, y coordinates
random.seed(42)
cities = {i: (random.randint(0, 10), random.randint(0, 10)) for i in range(5)}

def distance(a, b):
    """Calculate Euclidean distance between two cities."""
    (x1, y1), (x2, y2) = cities[a], cities[b]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

# 🧩 Helper: Compute total path length
def path_length(path):
    return sum(distance(path[i], path[i + 1]) for i in range(len(path) - 1)) + distance(path[-1], path[0])

# -------------------------------------
# 🚗 1. Nearest Neighbour Algorithm
# -------------------------------------
def nearest_neighbour(start=0):
    unvisited = list(cities.keys())
    unvisited.remove(start)
    path = [start]
    current = start

    while unvisited:
        next_city = min(unvisited, key=lambda c: distance(current, c))
        path.append(next_city)
        unvisited.remove(next_city)
        current = next_city

    return path, path_length(path)

# -------------------------------------
# 🪢 2. Multiple Fragment (Greedy edges)
# -------------------------------------
def multiple_fragment():
    edges = sorted(((distance(a, b), a, b) for a, b in itertools.combinations(cities, 2)))
    connections = {i: [] for i in cities}
    selected_edges = []

    for d, a, b in edges:
        if len(connections[a]) < 2 and len(connections[b]) < 2:
            if not creates_cycle(selected_edges, a, b, len(cities)):
                selected_edges.append((a, b))
                connections[a].append(b)
                connections[b].append(a)

    # Convert edge list into an ordered path
    path = build_path(selected_edges)
    return path, path_length(path)

def creates_cycle(edges, a, b, n):
    graph = {i: [] for i in range(n)}
    for x, y in edges:
        graph[x].append(y)
        graph[y].append(x)
    graph[a].append(b)
    graph[b].append(a)
    return has_cycle(graph)

def has_cycle(graph):
    visited = set()
    def dfs(node, parent):
        visited.add(node)
        for nei in graph[node]:
            if nei not in visited:
                if dfs(nei, node):
                    return True
            elif nei != parent:
                return True
        return False
    return any(dfs(node, -1) for node in graph if node not in visited)

def build_path(edges):
    graph = {i: [] for i in cities}
    for a, b in edges:
        graph[a].append(b)
        graph[b].append(a)
    start = next(k for k, v in graph.items() if len(v) == 1)
    path, visited = [start], {start}
    while len(path) < len(cities):
        next_city = next(c for c in graph[path[-1]] if c not in visited)
        path.append(next_city)
        visited.add(next_city)
    return path

# -------------------------------------
# 🌳 3. Branch & Bound (Brute-force version for simplicity)
# -------------------------------------
def brute_force():
    best_path, best_len = None, float("inf")
    for perm in itertools.permutations(cities.keys()):
        L = path_length(list(perm))
        if L < best_len:
            best_path, best_len = list(perm), L
    return best_path, best_len

# -------------------------------------
# 🧮 Run all methods
# -------------------------------------
nn_path, nn_len = nearest_neighbour()
mf_path, mf_len = multiple_fragment()
bf_path, bf_len = brute_force()

print("🗺 Cities:", cities)
print(f"🚗 Nearest Neighbour: {nn_path} -> {nn_len:.2f}")
print(f"🪢 Multiple Fragment: {mf_path} -> {mf_len:.2f}")
print(f"🌳 Brute Force (Best Route): {bf_path} -> {bf_len:.2f}")


🗺 Cities: {0: (10, 1), 1: (0, 4), 2: (3, 3), 3: (2, 1), 4: (10, 8)}
🚗 Nearest Neighbour: [0, 4, 2, 3, 1] -> 31.88
🪢 Multiple Fragment: [1, 2, 3, 0, 4] -> 31.17
🌳 Brute Force (Best Route): [1, 2, 4, 0, 3] -> 30.37


# Python simulation of the three TSP algorithms — Nearest Neighbour, Multiple Fragment, and Branch and Bound (simplified) — using a small map of 5 cities.

In [4]:
import itertools
import math

# Step 1: Define the cities and their coordinates
cities = {
    "A": (0, 0),
    "B": (1, 5),
    "C": (5, 2),
    "D": (6, 6),
    "E": (8, 3)
}

# Step 2: Helper function to calculate distance between two cities
def distance(city1, city2):
    x1, y1 = cities[city1]
    x2, y2 = cities[city2]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Step 3: Total route distance
def route_distance(route):
    dist = 0
    for i in range(len(route) - 1):
        dist += distance(route[i], route[i+1])
    dist += distance(route[-1], route[0])  # return home
    return dist

# 1. Nearest Neighbour Algorithm
def nearest_neighbour(start="A"):
    unvisited = set(cities.keys())
    route = [start]
    unvisited.remove(start)

    while unvisited:
        current = route[-1]
        next_city = min(unvisited, key=lambda city: distance(current, city))
        route.append(next_city)
        unvisited.remove(next_city)

    return route, route_distance(route)

# 2. Multiple Fragment Algorithm (simplified version)
def multiple_fragment():
    edges = []
    for a, b in itertools.combinations(cities.keys(), 2):
        edges.append((a, b, distance(a, b)))
    edges.sort(key=lambda x: x[2])

    connections = {city: [] for city in cities}
    route_edges = []

    for a, b, dist in edges:
        if len(connections[a]) < 2 and len(connections[b]) < 2:
            # Avoid small loops
            if not creates_small_loop(route_edges, a, b):
                connections[a].append(b)
                connections[b].append(a)
                route_edges.append((a, b))
            if len(route_edges) == len(cities):
                break

    # Build the route from the connections
    start = list(cities.keys())[0]
    route = [start]
    current = start
    prev = None
    while len(route) < len(cities):
        next_city = [x for x in connections[current] if x != prev][0]
        route.append(next_city)
        prev, current = current, next_city

    return route, route_distance(route)

def creates_small_loop(edges, a, b):
    # Simple check for small loops
    connected = {a}
    stack = [a]
    while stack:
        node = stack.pop()
        for x, y in edges:
            if node in (x, y):
                neighbor = y if node == x else x
                if neighbor not in connected:
                    connected.add(neighbor)
                    stack.append(neighbor)
    return b in connected and len(connected) < len(cities)

# 3. Branch and Bound (simplified: brute-force search)
def branch_and_bound():
    best_route = None
    best_distance = float("inf")

    for perm in itertools.permutations(cities.keys()):
        dist = route_distance(perm)
        if dist < best_distance:
            best_distance = dist
            best_route = perm

    return best_route, best_distance

# Step 4: Run the algorithms
nn_route, nn_dist = nearest_neighbour()
mf_route, mf_dist = multiple_fragment()
bb_route, bb_dist = branch_and_bound()

print("Nearest Neighbour Route:", nn_route, "Distance:", round(nn_dist, 2))
print("Multiple Fragment Route:", mf_route, "Distance:", round(mf_dist, 2))
print("Branch and Bound (Optimal) Route:", bb_route, "Distance:", round(bb_dist, 2))


Nearest Neighbour Route: ['A', 'B', 'C', 'E', 'D'] Distance: 25.35
Multiple Fragment Route: ['A', 'B', 'C', 'E', 'D'] Distance: 25.35
Branch and Bound (Optimal) Route: ('A', 'C', 'E', 'D', 'B') Distance: 22.35
